# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
# File to Load (Remember to Change These)
file_to_load = "../WeatherPy/weather_data.csv"

# Read File and store into Pandas data frame
weather_data = pd.read_csv(file_to_load)
weather_data1=weather_data.dropna()
weather_data1

,City,Lat,Lng,Max_Temp,Cloudiness,Humidity,Wind_Speed,country,date
0,busselton,-33.65,115.33,289.26,1.0,67.0,4.92,AU,1.603148e+09
1,kapaa,22.08,-159.32,302.59,40.0,78.0,7.20,US,1.603148e+09
2,khatanga,71.97,102.50,260.35,33.0,96.0,0.28,RU,1.603148e+09
3,brae,60.40,-1.35,281.15,100.0,87.0,17.00,GB,1.603148e+09
4,port elizabeth,-33.92,25.57,289.15,75.0,87.0,6.20,ZA,1.603147e+09
...,...,...,...,...,...,...,...,...,...
617,inirida,3.87,-67.92,299.30,87.0,79.0,0.73,CO,1.603147e+09
618,solnechnyy,50.72,136.63,272.43,0.0,76.0,2.15,RU,1.603148e+09
619,eureka,40.80,-124.16,289.26,90.0,93.0,4.30,US,1.603148e+09
620,corinto,37.94,22.96,289.26,55.0,70.0,2.15,GR,1.603148e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Store latitude and longitude in locations
locations = weather_data1[["Lat", "Lng"]]
locations
# Fill NaN values and convert to float
Humidity = weather_data1["Humidity"].astype(float)
Humidity

0      67.0
1      78.0
2      96.0
3      87.0
4      87.0
       ... 
617    79.0
618    76.0
619    93.0
620    70.0
621    45.0
Name: Humidity, Length: 564, dtype: float64

In [19]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# selecting units based on condition [A max temperature lower than 80 degrees but higher than 70.]
Max_Temp = weather_data1.loc[weather_data1["Max_Temp"] > 70] 
Max_Temp1 = Max_Temp.loc[Max_Temp["Max_Temp"] < 80]
print("A max temperature lower than 80 degrees but higher than 70.")
print(Max_Temp1)
print("---------------------------------------------------------------")

# selecting units based on condition [Wind speed less than 10 mph.]
Wind_speed = weather_data1.loc[weather_data1["Wind_Speed"] < 10]
print("Wind speed less than 10 mph.")
print(Wind_speed)
print("---------------------------------------------------------------")

# selecting units based on condition [Zero cloudiness.]
Cloudiness = weather_data1.loc[weather_data1["Cloudiness"] == 0]
print("Zero cloudiness.")
print(Cloudiness)
print("---------------------------------------------------------------")

# selecting units based on condition [ideal weather]
ideal_weather = Max_Temp1.loc[Max_Temp1["Wind_Speed"] < 10] 
ideal_weather1= ideal_weather.loc[ideal_weather["Cloudiness"] == 0]
print("ideal weather")
print(ideal_weather1)

A max temperature lower than 80 degrees but higher than 70.
Empty DataFrame
Columns: [City, Lat, Lng, Max_Temp, Cloudiness, Humidity, Wind_Speed, country, date]
Index: []
---------------------------------------------------------------
Wind speed less than 10 mph.
               City    Lat     Lng  Max_Temp  Cloudiness  Humidity  \
0         busselton -33.65  115.33    289.26         1.0      67.0   
1             kapaa  22.08 -159.32    302.59        40.0      78.0   
2          khatanga  71.97  102.50    260.35        33.0      96.0   
4    port elizabeth -33.92   25.57    289.15        75.0      87.0   
5           chicama  -7.84  -79.15    292.28        49.0      72.0   
..              ...    ...     ...       ...         ...       ...   
617         inirida   3.87  -67.92    299.30        87.0      79.0   
618      solnechnyy  50.72  136.63    272.43         0.0      76.0   
619          eureka  40.80 -124.16    289.26        90.0      93.0   
620         corinto  37.94   22.96  

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# geocoordinates
# We can get all the information from above.
target_coordinates=ideal_weather1[["Lat","Lng"]]
target_search = "hotels"
target_radius = 5000


# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

# find all hotels in those ideal cities.
counter = 0
print("The first Hotel in the seven cities with ideal weathers")
print("                                                        ")

hotels=[]
for place in places_data["results"]:
    hotels.append(place["name"])
    print(place["name"])
    print(place["vicinity"])
    print("--------------------------------")
    counter += 1
    if counter == len(target_coordinates):
        break
        
hotels

{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
The first Hotel in the seven cities with ideal weathers
                                                        


[]

In [22]:
hotel_df=ideal_weather1 
hotel_df["Hotel Name"] = hotels 
hotel_df

,City,Lat,Lng,Max_Temp,Cloudiness,Humidity,Wind_Speed,country,date,Hotel Name


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locationsA = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locationsA,
    info_box_content=[f"Hotel Map for ideal cities: {hotel}" for hotel in hotels])

fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))